<a href="https://colab.research.google.com/github/Kartik-Koul/102103819-SESS_LE1/blob/main/102103819_Kartik_Koul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Full Question with Table of Contents
_________________

1. Logistics
2. Task
3. Deliverables
4. Evaluation
5. Note
6. Struts


*Problem Code*: 202425ODD-UCS749-SESS-LE1-0911
*Problem Title*: Recognise My Voice Commands.


1. Logistics
===========

  + Start time: Wed Sep 11 10:00 AM
  + End Time: Wed Sep 11 04:00 PM
  + Submission Form:
    <https://docs.google.com/forms/d/e/1FAIpQLSc2aAG33E0rQgs3OsrpyjyREwy9OKK_WnKqmBLvD7pzM4sOvg/viewform?usp=pp_url&entry.185703634=202425ODD-UCS749-SESS-LE1-0911&entry.1322657816=Recognise+My+Voice+Commands>
  + Viva Voce: Will be notified later.


2. Task
======

  Consider the paper: <https://arxiv.org/abs/1804.03209>

  1. Read and summarise the paper in about 50 words.
  2. Download the dataset in the paper, statistically analyse and
     describe it, so that it may be useful for posterity. (Include code
     snippets in your .ipynb file to evidence your analysis.)
  3. Train a classifier so that you are able to distinguish the commands
     in the dataset.
  4. Report the performance results using standard benchmarks.
  5. Record about 30 samples of each command in your voice and create a
     new dataset (including a new user id for yourself).  You may use a
     timer on your computer to synchronise.
  6. Fine tune your classifier to perform on your voice.
  7. Report the results.


3. Deliverables
==============

  1. A PDF Report: (as a part of your Git Repo) named
     <ROLL_NO>-report.pdf
  2. Assets: Your pretrained classifier model weights and your cleaned
     and well-formed dataset.  This should be a part of your google
     drive with read access to your instructor <bv.raghav@thapar.edu>
  3. A demo notebook: (as a part of your Git Repo), that loads both your
     model and dataset; and runs to show the results.
  4. The demo notebook should verify the assets using a checksum
     (md5/sha/…).  This step verifies that the assets have not been
     tampered with at a later stage.


4. Evaluation
============

  1. Clarity of thought process and presentation.
  2. Data processing skills.
  3. Model fine tuning/ training skills.
  4. Details of progress, as in what were the encountered problems and
     how were they solved.
  5. How adaptable is your pipeline? (as in, how easy is it for me to
     adapt it for my voice)
  6. How scalable is your approach? (as in, how easy is it to scale it
     to many new voices)
  7. Strengths and Shortcomings of your approach.


5. Note
======

  1. This is a test of how fast can we report the performance of a model
     for a specific task.  The best performance is not expected; but a
     holistic pipeline is.
  2. You may improve upon it in future, out of interest; though it
     wouldn’t influence your eval.


6. Struts
========

  The following tutorials may be a good start point; there maybe more on
  the internet.  You are free to choose.
  1. <https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c64f4bad00653411821adcb75aea9015/speech_command_classification_with_torchaudio_tutorial.ipynb#scrollTo=i0pBRWkcxWrX>
  2. <https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/audio/simple_audio.ipynb>

## Initial model development and training with original dataset

In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.mkdir('/content/data')

In [ ]:
!tar -xzvf "//content/drive/MyDrive/speech_commands_v0.02.tar.gz" -C "/content/data" # Uploaded the dataset onto my drive

Streaming output truncated to the last 5000 lines.
./on/fc3ba625_nohash_0.wav
./on/db9d52d2_nohash_0.wav
./on/72242187_nohash_4.wav
./on/b0f5b16d_nohash_0.wav
./on/d9aa8c90_nohash_1.wav
./on/525eaa62_nohash_0.wav
./on/42e3f068_nohash_0.wav
./on/94d370bf_nohash_4.wav
./on/bf8d5617_nohash_0.wav
./on/e2b7d6d6_nohash_2.wav
./on/89ed36ab_nohash_0.wav
./on/30276d03_nohash_1.wav
./on/364c0853_nohash_0.wav
./on/7cbf645a_nohash_0.wav
./on/c98bbbd3_nohash_2.wav
./on/9a7c1f83_nohash_5.wav
./on/3589bc72_nohash_0.wav
./on/3ae5c04f_nohash_0.wav
./on/adc216c5_nohash_2.wav
./on/c50f55b8_nohash_15.wav
./on/324210dd_nohash_2.wav
./on/c120e80e_nohash_2.wav
./on/69086eb0_nohash_0.wav
./on/dce05676_nohash_1.wav
./on/435695e3_nohash_1.wav
./on/179a61b7_nohash_1.wav
./on/f5341341_nohash_0.wav
./on/91b03183_nohash_0.wav
./on/96d8bb6f_nohash_2.wav
./on/6a1908f8_nohash_0.wav
./on/6ace4fe1_nohash_1.wav
./on/84d1e469_nohash_1.wav
./on/ba676390_nohash_0.wav
./on/1dc86f91_nohash_4.wav
./on/3a929277_nohash_2.wav
./o

In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

In [ ]:
def load_files(folder_path, sample_rate=16000):
    audio_files = []
    labels = []
    class_names = sorted(os.listdir(folder_path))

    for class_name in class_names:
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            for file_name in os.listdir(class_folder):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(class_folder, file_name)
                    audio, _ = librosa.load(file_path, sr=sample_rate)

                    if len(audio) > sample_rate:
                        audio = audio[:sample_rate]
                    elif len(audio) < sample_rate:
                        padding = sample_rate - len(audio)
                        audio = np.pad(audio, (0, padding), 'constant')

                    audio = audio / np.max(np.abs(audio))

                    audio_files.append(audio)
                    labels.append(class_name)

    return np.array(audio_files), np.array(labels)

In [ ]:
train_dir = '/content/data'

audios, labels = load_files(train_dir)

print("Shape of audio array = ", audios.shape)
print("Shape of label array = ", labels.shape)

Shape of audio array =  (105835, 16000)
Shape of label array =  (105835,)


In [ ]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
train_audios, val_audios, train_labels, val_labels = train_test_split(
    audios, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

In [ ]:
print(f"Training set size: {train_audios.shape}")
print(f"Validation set size: {val_audios.shape}")

Training set size: (84668, 16000)
Validation set size: (21167, 16000)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_audios, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_audios, val_labels))

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = models.Sequential([
    layers.Input(shape=(16000,)),
    layers.Reshape((16000, 1)),

    layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=4),

    layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=4),

    layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=4),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 16000, 1)          0         
                                                                 
 conv1d (Conv1D)             (None, 16000, 32)         128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 4000, 32)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 4000, 64)          6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 1000, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 1000, 128)         2

In [ ]:
EPOCHS = 50

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print("Model training complete!")

Epoch 1/50
1323/1323 [==============================] - 210s 158ms/step - loss: 2.1801 - accuracy: 0.3659 - val_loss: 1.4783 - val_accuracy: 0.5721
Epoch 2/50
1323/1323 [==============================] - 206s 155ms/step - loss: 1.4638 - accuracy: 0.5660 - val_loss: 1.1668 - val_accuracy: 0.6659
Epoch 3/50
1323/1323 [==============================] - 207s 157ms/step - loss: 1.2114 - accuracy: 0.6377 - val_loss: 1.0404 - val_accuracy: 0.6961
Epoch 4/50
1323/1323 [==============================] - 210s 159ms/step - loss: 1.0372 - accuracy: 0.6837 - val_loss: 0.9979 - val_accuracy: 0.7105
Epoch 5/50
1323/1323 [==============================] - 207s 157ms/step - loss: 0.8965 - accuracy: 0.7202 - val_loss: 1.0527 - val_accuracy: 0.7035
Epoch 6/50
1323/1323 [==============================] - 205s 155ms/step - loss: 0.7816 - accuracy: 0.7535 - val_loss: 1.0363 - val_accuracy: 0.7176
Epoch 7/50
1323/1323 [==============================] - 201s 152ms/step - loss: 0.6751 - accuracy: 0.7826 - val_

In [ ]:
model.save('/content/drive/MyDrive/audio_recognition.h5')

print("Model saved successfully!")

Model saved successfully!


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Fine Tuning Portion with Custom Dataset

In [ ]:
custom_dataset_drive_path = '/content/drive/MyDrive/custom_dataset.zip'

In [ ]:
!unzip '/content/drive/MyDrive/custom_dataset.zip' -d '/content'

Archive:  /content/drive/MyDrive/custom_dataset.zip
   creating: /content/custom_dataset/
   creating: /content/custom_dataset/backward/
  inflating: /content/custom_dataset/backward/Recording (10).wav  
  inflating: /content/custom_dataset/backward/Recording (11).wav  
  inflating: /content/custom_dataset/backward/Recording (12).wav  
  inflating: /content/custom_dataset/backward/Recording (13).wav  
  inflating: /content/custom_dataset/backward/Recording (14).wav  
  inflating: /content/custom_dataset/backward/Recording (15).wav  
  inflating: /content/custom_dataset/backward/Recording (16).wav  
  inflating: /content/custom_dataset/backward/Recording (17).wav  
  inflating: /content/custom_dataset/backward/Recording (18).wav  
  inflating: /content/custom_dataset/backward/Recording (19).wav  
  inflating: /content/custom_dataset/backward/Recording (2).wav  
  inflating: /content/custom_dataset/backward/Recording (20).wav  
  inflating: /content/custom_dataset/backward/Recording (21)

In [ ]:
custom_dataset_path = '/content/custom_dataset'

In [ ]:
audios_cd, labels_cd = load_files(custom_dataset_path)

In [ ]:
print("Shape of audio data = ", audios_cd.shape)
print("Shape of label data = ", labels_cd.shape)

Shape of audio data =  (1055, 16000)
Shape of label data =  (1055,)


In [ ]:
label_encoder_cd = LabelEncoder()
labels_encoded_cd = label_encoder_cd.fit_transform(labels_cd)

print("Class names:", label_encoder_cd.classes_)

Class names: ['_background_noise_' 'backward' 'bed' 'bird' 'cat' 'dog' 'down' 'eight'
 'five' 'follow' 'forward' 'four' 'go' 'happy' 'house' 'learn' 'left'
 'marvin' 'nine' 'no' 'off' 'on' 'one' 'right' 'seven' 'sheila' 'six'
 'stop' 'three' 'tree' 'two' 'up' 'visual' 'wow' 'yes' 'zero']


In [ ]:
trained_model = tf.keras.models.load_model('/content/drive/MyDrive/audio_recognition (1).h5')

In [ ]:
for layer in trained_model.layers[:-2]:
  layer.trainable = False

In [ ]:
new_model = models.Sequential(trained_model.layers[:-2])

new_model.add(layers.Flatten(name='flatten_layer_finetune'))
new_model.add(layers.Dense(128, activation='relu', name='new_dense_layer_finetune'))
new_model.add(layers.Dropout(0.2, name='dropout_layer_finetune'))
new_model.add(layers.Dense(len(label_encoder_cd.classes_), activation='softmax', name='new_output_layer_finetune'))

new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

new_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 16000, 1)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 16000, 32)           │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 4000, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 4000, 64)            │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 1000, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 1000, 128)           │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 250, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,096,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_layer_finetune (Flatten)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ new_dense_layer_finetune (Dense)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_layer_finetune (Dropout)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ new_output_layer_finetune (Dense)    │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,127,168 (15.74 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,127,168 (15.74 MB)

In [ ]:
from sklearn.model_selection import train_test_split

train_audios_cd, val_audios_cd, train_labels_cd, val_labels_cd = train_test_split(
    audios_cd, labels_encoded_cd, test_size=0.15, random_state=42, stratify=labels_encoded_cd
)

In [ ]:
train_dataset_cd = tf.data.Dataset.from_tensor_slices((train_audios_cd, train_labels_cd))
val_dataset_cd = tf.data.Dataset.from_tensor_slices((val_audios_cd, val_labels_cd))

In [ ]:
BATCH_SIZE = 16
train_dataset_cd = train_dataset_cd.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_cd = val_dataset_cd.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
EPOCHS = 2000

history = new_model.fit(
    train_dataset_cd,
    epochs=EPOCHS,
    validation_data=val_dataset_cd
)

Epoch 1/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.0565 - loss: 4.6701 - val_accuracy: 0.2201 - val_loss: 2.8560
Epoch 2/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2736 - loss: 2.6166 - val_accuracy: 0.3585 - val_loss: 2.2501
Epoch 3/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4789 - loss: 1.8921 - val_accuracy: 0.4717 - val_loss: 1.8947
Epoch 4/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5531 - loss: 1.5495 - val_accuracy: 0.5535 - val_loss: 1.6890
Epoch 5/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6209 - loss: 1.2892 - val_accuracy: 0.5723 - val_loss: 1.5806
Epoch 6/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6905 - loss: 1.1295 - val_accuracy: 0.5975 - val_loss: 1.5336
Epoch 7/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7323 - loss: 0.9492 - val_accuracy: 0.5723 - val_loss: 1.4725
Epoch 8/2000
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7466 - loss: 0.8372 - val_accuracy: 0

In [ ]:
new_model.save('/content/drive/MyDrive/audio_recognition_finetune.h5')
print("Model has been fine-tuned and saved.")

Model has been fine-tuned and saved.


## Implementing Checksum

In [ ]:
import hashlib

def generate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

finetune_model_path = '/content/drive/MyDrive/audio_recognition_finetune.h5'

checksum = generate_md5(finetune_model_path)
print(f"MD5 checksum for {finetune_model_path}: {checksum}")

# Store the checksum in drive for future recognition purposes
with open('/content/drive/MyDrive/audio_recognition_checksum.md5', 'w') as f:
    f.write(checksum)

MD5 checksum for /content/drive/MyDrive/audio_recognition_finetune.h5: 849cebdb970a9a3a438d4bc87ffba7c5


In [ ]:
# Code for checksum verification of the file, to keep a check on tampering

def verify_md5(file_path, stored_checksum):
    computed_checksum = generate_md5(file_path)
    if computed_checksum == stored_checksum:
        print(f"Checksum verification passed for {file_path}.")
    else:
        print(f"Checksum verification failed for {file_path}!")

with open('/content/drive/MyDrive/audio_recognition_checksum.md5', 'r') as f:
    stored_checksum = f.read().strip()

verify_md5('/content/drive/MyDrive/audio_recognition_finetune.h5', stored_checksum)

Checksum verification passed for /content/drive/MyDrive/audio_recognition_finetune.h5.
